In [ ]:
######     The sentence are converted into the 300-D vectors
######      Sentence_Vector = the gravity center of the distributed word vectors in the sentence.

######                          \Sigma (word2vec(iword) * \rho(iword)) 
######      sen_vec    = ----------------------------------
######                             (\Sigma \rho(iword))
######
######       In this version2, \rho = tf-idf(iword)  while only words in word2vec (GoogleNews trained model) are calculated. 
######       tensorflow with 2 hidden layer

In [1]:
import gensim
import time
import numpy as np
import math
import os
import sys
import argparse
import random
from tempfile import gettempdir
import zipfile
import re
import csv
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# globalize the stopwords array
stopwords=['a','and','of','to','','\n']
global stopwords
filename_train='data/train.csv'
filename_test='data/test.csv'

In [4]:
# Step 1: Load the sentence in train data 

from itertools import islice

def load_sentence(filename_1,filename_2):
    
    text_content=[]
    file_1=open(filename_1,'r')
    for line in islice(file_1,1,None):    # SKIP the head line!!!
        text_content.append(line)
    file_1.close()   
    train_len=len(text_content)
    
    file_2=open(filename_2,"r")
    for line in islice(file_2,1,None):    # SKIP the head line!!!
        text_content.append(line)
    file_2.close()
    test_len=len(text_content) - train_len        
        
    return text_content,train_len,test_len                # NOTE THAT text_content: head line deleted!!!

text_content,train_len,test_len=load_sentence(filename_train,filename_test)        
        
print(len(text_content))
print(train_len+test_len)
type(text_content)

27971
27971


list

In [19]:
# Step 2: Get the ifidf value of words in each sentence ( including train and test data)

def create_tfidf(text_content):   # text_content : head line already deleted!
    
    tfidf=TfidfVectorizer(stop_words=stopwords)
    weight=tfidf.fit_transform(text_content).toarray()
    tf_vocabulary=tfidf.vocabulary_
    #iword=tf_vocabulary['leather']

    #print ('vocabulary list:\n')
    #for key,value in tfidf.vocabulary_.items():
    #    print (key,value)

    print('IFIDF weight matrix:\n')
    print (weight.shape)

    return tf_vocabulary,weight
since=time.time()
tf_voca,weight=create_tfidf(text_content)
print("Time consuming:", time.time()-since)
#iword=tf_voca['leather']

IFIDF weight matrix:

(27971, 56271)
Time consuming: 17.12557101249695


In [29]:
inverse_tf_voca=dict(zip(tf_voca.values(),tf_voca.keys()))
iword=tf_voca['survivor']
print(iword)
weight[588][iword]
for i in range(56270,56271):
    print(inverse_tf_voca[i])

52278
υπνος


In [18]:
# Step 3: Load the trained model: GoogleNews or Glove or whatever

since=time.time()
model = gensim.models.KeyedVectors.load_word2vec_format('/home/miaohan/trained_model/GoogleNews-vectors-negative300.bin', binary=True)
#new_sentence=['favour']
#model.train()
print(model['love'])
print("time:",time.time()-since)

[ 0.10302734 -0.15234375  0.02587891  0.16503906 -0.16503906  0.06689453
  0.29296875 -0.26367188 -0.140625    0.20117188 -0.02624512 -0.08203125
 -0.02770996 -0.04394531 -0.23535156  0.16992188  0.12890625  0.15722656
  0.00756836 -0.06982422 -0.03857422  0.07958984  0.22949219 -0.14355469
  0.16796875 -0.03515625  0.05517578  0.10693359  0.11181641 -0.16308594
 -0.11181641  0.13964844  0.01556396  0.12792969  0.15429688  0.07714844
  0.26171875  0.08642578 -0.02514648  0.33398438  0.18652344 -0.20996094
  0.07080078  0.02600098 -0.10644531 -0.10253906  0.12304688  0.04711914
  0.02209473  0.05834961 -0.10986328  0.14941406 -0.10693359  0.01556396
  0.08984375  0.11230469 -0.04370117 -0.11376953 -0.0037384  -0.01818848
  0.24316406  0.08447266 -0.07080078  0.18066406  0.03515625 -0.09667969
 -0.21972656 -0.00328064 -0.03198242  0.18457031  0.28515625 -0.0859375
 -0.11181641  0.0213623  -0.30664062 -0.09228516 -0.18945312  0.01513672
  0.18554688  0.34375    -0.31054688  0.22558594  0.

In [26]:
# Step 4: load the training file into training format

def load_train(filename,tf_voca,weight):   # Be Cautious : len(text in filename) = len(weight)+1  

    text_content=[]
    text_author=[]
    tf_word=[]
    input_file=open(filename,"r")
    iline=-1
    for line in islice(input_file,1,None):
        iline=iline+1
        row=re.split(" |!|\?|\.|\,|\"|\'|;|\:", line)    
        text_content=row[:]                 # Duplication cuz text_content could be removed in the row loop!
        if iline==588:
            print(text_content)

        
        for word in row:                      # strip the stop_words and meaningless str
            if word in stopwords:
                text_content.remove(word)      
        
        del text_content[0]                 # Cast the id item !
 
        if text_content[-1] == "EAP":
            #print("EAP HERE!")
            text_author.append([1,0,0])
        elif text_content[-1] == "HPL":
            text_author.append([0,1,0])
        elif text_content[-1] == "MWS":
            text_author.append([0,0,1])   #load Author
              
        del text_content[-1]                 # Cast the author name after adding to the one-hot text_author!      

        vec_sen_line=[0]*300     
        mass_sum=0    
        for word in text_content:        
            try:
                    
                word_index=tf_voca[word]             
                tfidf_value=weight[iline][word_index]
                if iline == 588:
                    print(text_content)
                    print("try",word,"iline",iline)
                    print("tfidf_value",tfidf_value)

                
                vec_sen_line=vec_sen_line+np.asarray(model[word]) * tfidf_value  # center of gravity= ({Sigma pos(i)*mass(i)}/{Sigma mass(i)})
                mass_sum=mass_sum+tfidf_value 
            except:
                pass
                #print("UKN word",word)
            #print("sum mass", mass_sum)
        #print("vec_sen_line type",type(vec_sen_line))
        if mass_sum != 0.0:
            
            tf_word.append(np.asarray(vec_sen_line)/mass_sum)
        elif mass_sum== 0.0:
            print("line",iline)
            tf_word.append(np.asarray(vec_sen_line))
     
    #print(text_id[0],text_author[0],tf_word[0])
    #print(len(text_id),len(text_author),len(tf_word))
   
    print("file loaded!")
    input_file.close()
    return text_author,tf_word


since=time.time()
text_author,tf_word=load_train(filename_train,tf_voca,weight)
print("computation time:",time.time()-since)

['', 'id08064', '', '', 'One', 'Survivor', 'and', 'Dead', 'Man', 'Found', 'Aboard', '', '', '', 'HPL', '\n']
line 588
line 2304
line 4558
line 6262
line 6647
line 6936
line 9250
line 9950
line 10542
line 11306
line 13401
line 14012
line 18064
line 18127
line 18501
file loaded!
computation time: 3.308793067932129


In [64]:

# Test the itertools -- islice

from itertools import islice
input_file = open("data/test.txt")
for line in islice(input_file, 1, None):
    print(line)
    

"id12354","His head of hair would have done honor to a Brutus; nothing could be more richly flowing, or possess a brighter gloss.","MWS"

"id12465",""They shout," I said, "because they will soon return to England." "Do you, then, really return?" "Alas Yes; I cannot withstand their demands.","HPL"



In [31]:
def load_test(filename,train_len,tf_voca,weight):
    text_id=[]
    text_content=[]
    tf_word=[]
    
    file=open(filename,"r")
    iline=-1
    for line in islice(file,1,None):
        iline=iline+1
        #print("line",line)
        row=re.split(" |!|\?|\.|\,|\"|\'|;|\:", line)
        text_content=row[:]

        # strip the stop_words and meaningless str

        for word in row:  
            if word in stopwords:
                
                text_content.remove(word)
        #print(text_content)        
        
        
        text_id.append(text_content[0]) # load id
        #print("text_content[0]",text_content[0])
        del text_content[0]
 

                       
        vec_sen_line=[0]*300     
        mass_sum=0    
        for word in text_content:        
            try:
                word_index=tf_voca[word]
                tfidf_value=weight[iline+train_len][word_index]
                
                vec_sen_line=vec_sen_line+np.asarray(model[word]) *tfidf_value # center of gravity= ({Sigma pos(i)*mass(i)}/{Sigma mass(i)})
                mass_sum=mass_sum+tfidf_value
            except:
                pass
                #print("UKN word",word)
                
        #print("vec_sen_line type",type(vec_sen_line))
        if mass_sum != 0.0  :
            tf_word.append(np.asarray(vec_sen_line)/mass_sum)
        elif mass_sum == 0.0:
            print("iline",iline)
            tf_word.append(np.asarray(vec_sen_line))
     
 
    print("file loaded!")
    file.close()
    return text_id,tf_word


test_id,test_word=load_test(filename_test,train_len,tf_voca,weight)

iline 439
iline 1011
iline 2777
iline 4056
iline 4081
iline 5068
iline 5792
iline 8097
file loaded!


In [33]:
def train_test(text_author,tf_word):
    train_author,train_word=text_author[0:18000],tf_word[0:18000]
    test_author,test_word=text_author[18000:],tf_word[18000:]

    return train_word,train_author,test_word,test_author
a=[]
a.append(['Then', 'I', 'told', 'him', 'what', 'I'])
a.append(['scars', 'on', 'my', "ancestor", 'chest'])


train_x,train_y,test_x,test_y = train_test(text_author,tf_word)
print(len(train_x))
print(len(test_x))
#print(sen_vec)
#print(len(sen_vec))
#print(sen_vec[0][0])





18000
1579


In [34]:
def save_file(filename,tid,prediction):
    import csv
    with open(filename,"w") as csvfile: 
        writer = csv.writer(csvfile)

        #columns_name
        writer.writerow(["id","EAP","HPL","MWS"])
        #写入多行用writerows
        for ii in range(len(tid)):
            type(prediction[ii])
            a=[tid[ii]]+list(prediction[ii])
            writer.writerow(a)
            
            

In [35]:
######     2 hidden layers    #####

from __future__ import print_function 
import tensorflow as tf

#mnist = input_data.read_data_sets("mnist_data/", one_hot=True) 

# Parameters
learning_rate = 0.01
training_epochs = 100
batch_size = 20
display_step = 10
print (len(tf_word))  #19579

###### tf Graph Input #####
x = tf.placeholder(tf.float32, [None, 300]) # word2vec of shape 300 features
y = tf.placeholder(tf.float32, [None, 3]) # 3 authors to be classified => 3 classes

n_hidden_1 = 20 # 1st layer number of neurons
n_hidden_2 = 5 # 2nd layer number of neurons
n_input = 300 # word2vec of shape 300 features
n_classes = 3 # authors classes 3

##### Set model weights #####
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}
W = tf.Variable(tf.zeros([300, 3]))
b = tf.Variable(tf.zeros([3]))

# Create model
def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer


# Construct multi-layers model
#logits = multilayer_perceptron(x)
#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))


pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
#print(pred.get_shape())

cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))

##### Gradient Descent #####
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

##### Initializing the variables #####
init = tf.global_variables_initializer()



19579


In [37]:
##### Launch the graph #####
with tf.Session() as sess:
    sess.run(init)
 
     ### Training cycle ###
    for epoch in range(training_epochs):
            
        avg_cost = 0.
        total_batch = int(len(train_x)/batch_size)
        print("epoch",epoch)
        print("total_batch",total_batch)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = train_x[i*batch_size:(i+1)*batch_size], train_y[i*batch_size : (i+1)*batch_size]
              # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                           y: batch_ys})
            # Compute average loss
            avg_cost += c / total_batch
         # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
 
    print("Optimization Finished!")

    
    ##### Test model#####
 #   pred=tf.nn.softmax(logits)
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1)) 
    print(pred.get_shape())
    print(y.get_shape())
    ##### Calculate accuracy #####     
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: test_x, y: test_y}))
    
    
    print("submission", pred.eval({x:test_word}))
    prediction=pred.eval({x:test_word})
    #prediction.get_shape()
    filename_save='subm2.1_2lys.csv'
    save_file(filename_save,test_id,prediction)

epoch 0
total_batch 900
epoch 1
total_batch 900
epoch 2
total_batch 900
epoch 3
total_batch 900
epoch 4
total_batch 900
epoch 5
total_batch 900
epoch 6
total_batch 900
epoch 7
total_batch 900
epoch 8
total_batch 900
epoch 9
total_batch 900
Epoch: 0010 cost= 0.770758537
epoch 10
total_batch 900
epoch 11
total_batch 900
epoch 12
total_batch 900
epoch 13
total_batch 900
epoch 14
total_batch 900
epoch 15
total_batch 900
epoch 16
total_batch 900
epoch 17
total_batch 900
epoch 18
total_batch 900
epoch 19
total_batch 900
Epoch: 0020 cost= 0.769894322
epoch 20
total_batch 900
epoch 21
total_batch 900
epoch 22
total_batch 900
epoch 23
total_batch 900
epoch 24
total_batch 900
epoch 25
total_batch 900
epoch 26
total_batch 900
epoch 27
total_batch 900
epoch 28
total_batch 900
epoch 29
total_batch 900
Epoch: 0030 cost= 0.769804073
epoch 30
total_batch 900
epoch 31
total_batch 900
epoch 32
total_batch 900
epoch 33
total_batch 900
epoch 34
total_batch 900
epoch 35
total_batch 900
epoch 36
total_batch